# NLP for Full_Text
coba biar lebih rapi

## Import Library

In [2]:
# ===== Import Library =====
import re
import json
import pandas as pd
from tqdm import tqdm
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import nltk

# pastikan nltk stopwords tersedia
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load Dataset & Path

In [3]:
# ===== Load & Clean Dataset =====
import pandas as pd

# Path file
file_path = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\Cleaned_1.csv'
output_path = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\Cleaned_DropNull.csv'

# 1. Load dataset
df = pd.read_csv(file_path)
print("✅ Dataset Loaded Successfully")
print(f"Jumlah baris awal: {len(df)}")
print(f"Kolom tersedia: {list(df.columns)}")

# 2. Cari baris invalid/null
invalid_rows = df[
    df['image_corelation'].isna() | (df['image_corelation'] == '-') |
    df['Label'].isna() | (df['Label'] == '-')
]

print("\n🔍 Pengecekan data invalid:")
print("Jumlah baris invalid unik:", len(invalid_rows))

# 3. Mask valid rows
mask = (
    df['image_corelation'].notna() & (df['image_corelation'] != '-') &
    df['Label'].notna() & (df['Label'] != '-')
)

# 4. Terapkan mask → dataset cleaned
df_cleaned = df[mask].copy()

print("\n📊 Ringkasan pembersihan:")
print("Jumlah baris sebelum drop:", len(df))
print("Jumlah baris sesudah drop:", len(df_cleaned))
print("Jumlah baris yang dihapus:", len(df) - len(df_cleaned))

# 5. Simpan dataset hasil cleaning
df_cleaned.to_csv(output_path, index=False)
print(f"\n💾 Dataset cleaned berhasil disimpan ke {output_path}")

# Preview
df_cleaned.head(5)


✅ Dataset Loaded Successfully
Jumlah baris awal: 18450
Kolom tersedia: ['conversation_id_str', 'created_at', 'favorite_count', 'full_text', 'id_str', 'image_url', 'in_reply_to_screen_name', 'lang', 'location', 'quote_count', 'reply_count', 'retweet_count', 'tweet_url', 'user_id_str', 'username', 'official_account', 'age_account', 'verified_account', 'followers_count', 'image_corelation', 'Label', 'MM/DD/YYYY']

🔍 Pengecekan data invalid:
Jumlah baris invalid unik: 1924

📊 Ringkasan pembersihan:
Jumlah baris sebelum drop: 18450
Jumlah baris sesudah drop: 16526
Jumlah baris yang dihapus: 1924

💾 Dataset cleaned berhasil disimpan ke E:\$7th\TA\Multimodal_Process_Exploration\DATA\Cleaned_DropNull.csv


,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,...,tweet_url,user_id_str,username,official_account,age_account,verified_account,followers_count,image_corelation,Label,MM/DD/YYYY
0,1874825987416109464,Thu Jan 02 14:32:02 +0000 2025,494,1. Mulyono finalis tokoh terkorup dunia 2024 2...,1874825987416109464,https://pbs.twimg.com/ext_tw_video_thumb/18748...,NaN,in,NaN,3,...,https://x.com/undefined/status/187482598741610...,1159689173340585984,__AnakKolong,1,6 Tahun 8 Bulan,1,91600.0,0,1,08/01/2019
1,1894326707387338853,Tue Feb 25 10:00:56 +0000 2025,0,Saham indo bluchip ya bagus2 aja cuma pair nya...,1894326707387338853,https://pbs.twimg.com/media/GkoAGfWbkAYpK7_.jpg,NaN,in,NaN,1,...,https://x.com/undefined/status/189432670738733...,2801067998,__Anto_77_,0,11 Tahun 8 Bulan,0,695.0,1,0,09/01/2014
2,1895397644664705226,Fri Feb 28 08:56:27 +0000 2025,0,Pejuang Rupiah! https://t.co/MyCuEUKTfP,1895397644664705226,https://pbs.twimg.com/media/Gk3OGjPaoAAvMJF.jpg,NaN,in,NaN,0,...,https://x.com/undefined/status/189539764466470...,1848923371222011904,__BADjingan,0,1 Tahun 8 Bulan,0,133.0,0,0,10/01/2024
3,1902708493775491221,2025-03-20 13:07:10,0,IHSG menuju 5000 https://t.co/6FlyFblaMt,1902708493775491221,https://pbs.twimg.com/media/GmfHSZiagAAycIu.jpg,NaN,in,NaN,0,...,https://x.com/undefined/status/190270849377549...,442517900,__badkisser,0,13 Tahun 9 Bulan,0,353.0,1,0,2011-12-01
4,1885692367984054742,Sat Feb 01 14:11:09 +0000 2025,23,Rupiah Mendadak Perkasa ada dilevel Rp 8170 Pe...,1885692367984054742,https://pbs.twimg.com/media/GitTNLsaAAApkEu.jpg,NaN,in,NaN,0,...,https://x.com/undefined/status/188569236798405...,799023207294504960,__LOVE_AG4EVER,1,9 Tahun 8 Bulan,1,128000.0,1,1,11/01/2016


## Cleaning

In [4]:
# ===== Cleaning =====
def cleaning_text(text):
    text = re.sub(r'http\S+|www\S+', '', text)                 # hapus URL
    text = re.sub(r'@\w+|#\w+', '', text)                      # hapus mention & hashtag
    text = re.sub(r'(\d+)\s*-\s*(\d+)', r'\1_\2', text)        # ganti range angka jadi 3_5
    text = re.sub(r'\b[a-zA-Z]\b', '', text)                   # hapus kata 1 huruf
    text = re.sub(r'-\b', '', text)                            # hilangkan strip di akhir kata
    text = re.sub(r'[^a-zA-Z0-9\s_]', ' ', text)               # hapus simbol/emoji
    text = re.sub(r'\s+', ' ', text).strip()                   # rapikan spasi
    return text

tqdm.pandas(desc="Cleaning")
df_cleaned['cleaned'] = df_cleaned['full_text'].progress_apply(cleaning_text)

df_cleaned[['full_text','cleaned']].head(5)


Cleaning: 100%|██████████| 16526/16526 [00:00<00:00, 41343.90it/s]


,full_text,cleaned
0,1. Mulyono finalis tokoh terkorup dunia 2024 2...,1 Mulyono finalis tokoh terkorup dunia 2024 2 ...
1,Saham indo bluchip ya bagus2 aja cuma pair nya...,Saham indo bluchip ya bagus2 aja cuma pair nya...
2,Pejuang Rupiah! https://t.co/MyCuEUKTfP,Pejuang Rupiah
3,IHSG menuju 5000 https://t.co/6FlyFblaMt,IHSG menuju 5000
4,Rupiah Mendadak Perkasa ada dilevel Rp 8170 Pe...,Rupiah Mendadak Perkasa ada dilevel Rp 8170 Pe...


## Case Folding

In [5]:
# ===== Case Folding =====
def case_folding(text):
    return text.lower()

tqdm.pandas(desc="Case Folding")
df_cleaned['casefold'] = df_cleaned['cleaned'].progress_apply(case_folding)

df_cleaned[['cleaned','casefold']].head(5)


Case Folding: 100%|██████████| 16526/16526 [00:00<00:00, 1090699.88it/s]


,cleaned,casefold
0,1 Mulyono finalis tokoh terkorup dunia 2024 2 ...,1 mulyono finalis tokoh terkorup dunia 2024 2 ...
1,Saham indo bluchip ya bagus2 aja cuma pair nya...,saham indo bluchip ya bagus2 aja cuma pair nya...
2,Pejuang Rupiah,pejuang rupiah
3,IHSG menuju 5000,ihsg menuju 5000
4,Rupiah Mendadak Perkasa ada dilevel Rp 8170 Pe...,rupiah mendadak perkasa ada dilevel rp 8170 pe...


## Tokenization

In [6]:
# ===== Tokenization =====
def tokenize_text(text):
    return text.split()

tqdm.pandas(desc="Tokenization")
df_cleaned['tokenized'] = df_cleaned['casefold'].progress_apply(tokenize_text)

df_cleaned[['casefold','tokenized']].head(5)


Tokenization: 100%|██████████| 16526/16526 [00:00<00:00, 344323.46it/s]


,casefold,tokenized
0,1 mulyono finalis tokoh terkorup dunia 2024 2 ...,"[1, mulyono, finalis, tokoh, terkorup, dunia, ..."
1,saham indo bluchip ya bagus2 aja cuma pair nya...,"[saham, indo, bluchip, ya, bagus2, aja, cuma, ..."
2,pejuang rupiah,"[pejuang, rupiah]"
3,ihsg menuju 5000,"[ihsg, menuju, 5000]"
4,rupiah mendadak perkasa ada dilevel rp 8170 pe...,"[rupiah, mendadak, perkasa, ada, dilevel, rp, ..."


## Normalization

In [7]:
# ===== Normalisasi Slang =====
slang_path = r"E:\$7th\TA\Multimodal_Process_Exploration\Kamus\combined_slang_words.json"
with open(slang_path,'r') as f:
    slang_dict = json.load(f)

def normalize_tokens(tokens, slang_dict):
    return [slang_dict.get(w,w) for w in tokens]

tqdm.pandas(desc="Normalization")
df_cleaned['normalized'] = df_cleaned['tokenized'].progress_apply(lambda x: normalize_tokens(x, slang_dict))

df_cleaned[['tokenized','normalized']].head(5)


Normalization: 100%|██████████| 16526/16526 [00:00<00:00, 290724.30it/s]


,tokenized,normalized
0,"[1, mulyono, finalis, tokoh, terkorup, dunia, ...","[1, mulyono, finalis, tokoh, terkorup, dunia, ..."
1,"[saham, indo, bluchip, ya, bagus2, aja, cuma, ...","[saham, indonesia, bluchip, iya, bagus-bagus, ..."
2,"[pejuang, rupiah]","[pejuang, rupiah]"
3,"[ihsg, menuju, 5000]","[ihsg, menuju, 5000]"
4,"[rupiah, mendadak, perkasa, ada, dilevel, rp, ...","[rupiah, mendadak, perkasa, ada, dilevel, rp, ..."


## Stopwords Removal

In [8]:
# ===== Stopword Removal =====
stopwords_path = r"E:\$7th\TA\Multimodal_Process_Exploration\Kamus\combined_stop_words.txt"
with open(stopwords_path,'r') as f:
    custom_stop = set([line.strip() for line in f])

nltk_stop = set(stopwords.words('indonesian'))
all_stop = nltk_stop.union(custom_stop)

def remove_stopwords(tokens, stopset):
    return [w for w in tokens if w not in stopset]

tqdm.pandas(desc="Stopword Removal")
df_cleaned['stop_removed'] = df_cleaned['normalized'].progress_apply(lambda x: remove_stopwords(x, all_stop))

df_cleaned[['normalized','stop_removed']].head(5)


Stopword Removal: 100%|██████████| 16526/16526 [00:00<00:00, 313285.61it/s]


,normalized,stop_removed
0,"[1, mulyono, finalis, tokoh, terkorup, dunia, ...","[1, mulyono, finalis, tokoh, terkorup, dunia, ..."
1,"[saham, indonesia, bluchip, iya, bagus-bagus, ...","[saham, indonesia, bluchip, iya, bagus-bagus, ..."
2,"[pejuang, rupiah]","[pejuang, rupiah]"
3,"[ihsg, menuju, 5000]","[ihsg, 5000]"
4,"[rupiah, mendadak, perkasa, ada, dilevel, rp, ...","[rupiah, mendadak, perkasa, dilevel, rp, 8170,..."


## Stemming

In [9]:
# ===== Stemming =====
exceptions_path = r"E:\$7th\TA\Multimodal_Process_Exploration\Kamus\combined_exceptions.txt"
with open(exceptions_path,'r') as f:
    exception_list = set([line.strip() for line in f if line.strip()])

stemmer = StemmerFactory().create_stemmer()

def stemming_tokens(tokens, stemmer, exceptions):
    result = []
    for w in tokens:
        if w in exceptions:
            result.append(w)
        else:
            result.append(stemmer.stem(w))
    return result

tqdm.pandas(desc="Stemming")
df_cleaned['stemmed'] = df_cleaned['stop_removed'].progress_apply(lambda x: stemming_tokens(x, stemmer, exception_list))

df_cleaned[['stop_removed','stemmed']].head(5)


Stemming: 100%|██████████| 16526/16526 [25:05<00:00, 10.98it/s] 


,stop_removed,stemmed
0,"[1, mulyono, finalis, tokoh, terkorup, dunia, ...","[1, mulyono, final, tokoh, korup, dunia, 2024,..."
1,"[saham, indonesia, bluchip, iya, bagus-bagus, ...","[saham, indonesia, bluchip, iya, bagus, pair, ..."
2,"[pejuang, rupiah]","[juang, rupiah]"
3,"[ihsg, 5000]","[ihsg, 5000]"
4,"[rupiah, mendadak, perkasa, dilevel, rp, 8170,...","[rupiah, dadak, perkasa, level, rp, 8170, dola..."


## Detokenize

In [14]:
# ===== Detokenize =====
from nltk.tokenize.treebank import TreebankWordDetokenizer

def detokenize(tokens):
    return TreebankWordDetokenizer().detokenize(tokens)

tqdm.pandas(desc="Detokenize")
df_cleaned['clean_text'] = df_cleaned['stemmed'].progress_apply(detokenize)

df_cleaned[['full_text','clean_text']].head(5)


Detokenize: 100%|██████████| 16526/16526 [00:00<00:00, 20105.76it/s]


,full_text,clean_text
0,1. Mulyono finalis tokoh terkorup dunia 2024 2...,1 mulyono final tokoh korup dunia 2024 2 negar...
1,Saham indo bluchip ya bagus2 aja cuma pair nya...,saham indonesia bluchip iya bagus pair nya jel...
2,Pejuang Rupiah! https://t.co/MyCuEUKTfP,juang rupiah
3,IHSG menuju 5000 https://t.co/6FlyFblaMt,ihsg 5000
4,Rupiah Mendadak Perkasa ada dilevel Rp 8170 Pe...,rupiah dadak perkasa level rp 8170 dolar as ca...


## Save Final File

In [15]:
# ===== Save Final Result =====
cleaned_text_path = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\Cleaned_Text.csv'
save_copy = r'E:\$7th\TA\Multimodal_Process_Exploration\DATA\Cleaned_Text_Backup.csv'

susunan_kolom = [
    'full_text',
    'clean_text',
    'tweet_url',
    'image_url',
    'user_id_str',
    'conversation_id_str',
    'created_at',
    'MM/DD/YYYY',
    'age_account',
    'username',
    'official_account',
    'verified_account',
    'quote_count',
    'reply_count',
    'retweet_count',
    'favorite_count',
    'followers_count',
    'image_corelation',
    'Label'
]

#susunan_kolom = ['full_text','clean_text','image_corelation','Label']
final_df = df_cleaned[susunan_kolom]

final_df.to_csv(cleaned_text_path, index=False)
final_df.to_csv(save_copy, index=False)

print(f"Preprocessing selesai! Hasil disimpan di: {cleaned_text_path}")

final_df.head(5)


Preprocessing selesai! Hasil disimpan di: E:\$7th\TA\Multimodal_Process_Exploration\DATA\Cleaned_Text.csv


,full_text,clean_text,tweet_url,image_url,user_id_str,conversation_id_str,created_at,MM/DD/YYYY,age_account,username,official_account,verified_account,quote_count,reply_count,retweet_count,favorite_count,followers_count,image_corelation,Label
0,1. Mulyono finalis tokoh terkorup dunia 2024 2...,1 mulyono final tokoh korup dunia 2024 2 negar...,https://x.com/undefined/status/187482598741610...,https://pbs.twimg.com/ext_tw_video_thumb/18748...,1159689173340585984,1874825987416109464,Thu Jan 02 14:32:02 +0000 2025,08/01/2019,6 Tahun 8 Bulan,__AnakKolong,1,1,3,66,138,494,91600.0,0,1
1,Saham indo bluchip ya bagus2 aja cuma pair nya...,saham indonesia bluchip iya bagus pair nya jel...,https://x.com/undefined/status/189432670738733...,https://pbs.twimg.com/media/GkoAGfWbkAYpK7_.jpg,2801067998,1894326707387338853,Tue Feb 25 10:00:56 +0000 2025,09/01/2014,11 Tahun 8 Bulan,__Anto_77_,0,0,1,1,0,0,695.0,1,0
2,Pejuang Rupiah! https://t.co/MyCuEUKTfP,juang rupiah,https://x.com/undefined/status/189539764466470...,https://pbs.twimg.com/media/Gk3OGjPaoAAvMJF.jpg,1848923371222011904,1895397644664705226,Fri Feb 28 08:56:27 +0000 2025,10/01/2024,1 Tahun 8 Bulan,__BADjingan,0,0,0,0,0,0,133.0,0,0
3,IHSG menuju 5000 https://t.co/6FlyFblaMt,ihsg 5000,https://x.com/undefined/status/190270849377549...,https://pbs.twimg.com/media/GmfHSZiagAAycIu.jpg,442517900,1902708493775491221,2025-03-20 13:07:10,2011-12-01,13 Tahun 9 Bulan,__badkisser,0,0,0,0,0,0,353.0,1,0
4,Rupiah Mendadak Perkasa ada dilevel Rp 8170 Pe...,rupiah dadak perkasa level rp 8170 dolar as ca...,https://x.com/undefined/status/188569236798405...,https://pbs.twimg.com/media/GitTNLsaAAApkEu.jpg,799023207294504960,1885692367984054742,Sat Feb 01 14:11:09 +0000 2025,11/01/2016,9 Tahun 8 Bulan,__LOVE_AG4EVER,1,1,0,2,12,23,128000.0,1,1
